In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier

In [2]:
# Параметры-константы для сопоставимости результатов моделей:
SEED = 7
NUM_TREES = 30

In [3]:
# Данные для модели:
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [4]:
# Индексы для деления данных на 10 частей, т.к. исходные данные не перемешаны:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=SEED)

In [5]:
# Создаем модели:
model_ABC = AdaBoostClassifier(n_estimators=NUM_TREES, random_state=SEED)
model_RFC = RandomForestClassifier(n_estimators=NUM_TREES, random_state=SEED)
model_GBC = GradientBoostingClassifier(n_estimators=NUM_TREES, random_state=SEED)

In [6]:
models = [model_ABC, model_RFC, model_GBC]  # Список моделей для итерирования

In [7]:
def cross_val_accuracy(model, cv=skf, scoring='accuracy'):
    """Функция принимает модель классификации, выводит
    параметры модели и показатели точности при кросс-валидации."""
    print(f'\nПараметры модели:\n{model}')
    results = cross_val_score(model, X, y, cv=cv, scoring=scoring)
    print(f'\nСредняя точность модели при кросс-валидации: {results.mean()}')
    for ind, accuracy in enumerate(results):
        print(f'\tВыборка №{ind+1}:\tТочность: {accuracy}')

In [8]:
def test_accuracy(model):
    """Функция принимает и обучает модель классификации, делает прогноз
    и выводит показатели точности модели на тестовой выборке."""
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)
    accuracy = accuracy_score(y_test, prediction)
    print(f'Точность модели на тестовой выборке: {accuracy}')

In [9]:
# Последовательно оцениваем точность моделей:
for model in models:
    cross_val_accuracy(model)  # при кросс-валидации
    test_accuracy(model)  # на тестовой выборке


Параметры модели:
AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=30, random_state=7)

Средняя точность модели при кросс-валидации: 0.9333333333333333
	Выборка №1:	Точность: 0.9333333333333333
	Выборка №2:	Точность: 0.8666666666666667
	Выборка №3:	Точность: 0.9333333333333333
	Выборка №4:	Точность: 0.9333333333333333
	Выборка №5:	Точность: 1.0
	Выборка №6:	Точность: 1.0
	Выборка №7:	Точность: 0.9333333333333333
	Выборка №8:	Точность: 0.8
	Выборка №9:	Точность: 0.9333333333333333
	Выборка №10:	Точность: 1.0
Точность модели на тестовой выборке: 0.8666666666666667

Параметры модели:
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0,

In [10]:
def optimize_model(model, params, cv=skf, scoring='accuracy'):
    """Функция находит оптимальные параметры модели
    через настройку гиперпараметров RandomizedSearchCV.
    Принимает модель и словарь параметров.
    Выводит параметры лучшей модели и возвращает ее."""
    print(f'\nНастройка модели:\n{model}')
    model_tuned = RandomizedSearchCV(model, param_distributions=params,
                                     n_iter=10, cv=skf,
                                     scoring='accuracy',
                                     random_state=SEED,
                                     refit=True,
                                     n_jobs=-1)
    model_tuned.fit(X, y)
    print('Параметры оптимальной модели:', model_tuned.best_params_)
    print('Точность оптимальной модели:', model_tuned.best_score_)
    return model_tuned

In [11]:
# Словари параметров для моделей:
params_ABC = {'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
              'learning_rate': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
              'algorithm': ['SAMME', 'SAMME.R']}

In [12]:
params_RFC = {'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
              'criterion': ['gini', 'entropy'],
              'max_depth': [3, 5, 10, None],
              'min_samples_split': [2, 3, 4],
              'min_samples_leaf': [1, 2, 3],
              'max_features': ['auto', None],
              'bootstrap': [True, False]}

In [13]:
params_GBC = {'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
              'learning_rate': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
              'subsample': [0.8, 0.9, 1.0],
              'min_samples_split': [2, 3, 4],
              'min_samples_leaf': [1, 2, 3],
              'max_depth': [3, 5, 10, None],
              'max_features': ['auto', None]}

In [14]:
params = [params_ABC, params_RFC, params_GBC]  # Список для итерирования

In [15]:
models_tuned = []  # Список для добавления оптимальных моделей
for model, pars in zip(models, params):
    models_tuned.append(optimize_model(model, pars))


Настройка модели:
AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=30, random_state=7)
Параметры оптимальной модели: {'n_estimators': 30, 'learning_rate': 0.3, 'algorithm': 'SAMME'}
Точность оптимальной модели: 0.9533333333333334

Настройка модели:
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=30,
                       n_jobs=None, oob_score=False, random_state=7, verbose=0,
                       warm_start=False)
Параметры оптимальной модели: {'n_estimators': 80, 'min_samples_split': 4, 'min_samples_leaf': 3, 'max_features': None, 'max_depth': 10, 'criterion': 'gini', 'bootstrap': True}
Точность оптимальной мо